In [3]:
import os
import pandas as pd
import numpy as np
import pretty_midi
from music21 import converter

ModuleNotFoundError: No module named 'music21'

In [ ]:
import os
import pickle
import pretty_midi
from music21 import converter

# Directorio base donde están los archivos organizados por artista
base_dir = './DBN/dataset/'  # Ajusta esta ruta según corresponda

# Diccionario donde se guardará la información
# Cada entrada tendrá la forma: { track_id: { 'artist': ..., 'track': ..., 'composer': ..., 'key': ..., 'midi_path': ..., 'piano_roll': ... } }
dataset_dict = {}
track_id = 0

# Recorrer cada carpeta (artista) y cada archivo MIDI dentro
for artist in os.listdir(base_dir):
    artist_path = os.path.join(base_dir, artist)
    if os.path.isdir(artist_path):
        for file_name in os.listdir(artist_path):
            if file_name.lower().endswith(('.mid', '.midi')):
                midi_path = os.path.join(artist_path, file_name)
                
                # Extraer metadatos usando music21
                try:
                    score = converter.parse(midi_path)
                except Exception as e:
                    print(f"Error al parsear {midi_path}: {e}")
                    continue

                # Obtener compositor (si está disponible)
                composer = score.metadata.composer if score.metadata is not None else None

                # Extraer la tonalidad
                try:
                    key_obj = score.analyze('key')
                    key_str = f"{key_obj.tonic.name} {key_obj.mode}"
                except Exception as e:
                    key_str = None

                # Extraer el piano roll usando pretty_midi
                try:
                    pm = pretty_midi.PrettyMIDI(midi_path)
                    # Genera un piano roll de todas las pistas (dimensiones: 128 x time_steps)
                    piano_roll = pm.get_piano_roll(fs=100)  # fs=100 indica frames por segundo
                except Exception as e:
                    print(f"Error al obtener piano roll de {midi_path}: {e}")
                    continue

                # Guardar la información en el diccionario
                dataset_dict[track_id] = {
                    'artist': artist,
                    'track': file_name,
                    'composer': composer,
                    'key': key_str,
                    'midi_path': midi_path,
                    'piano_roll': piano_roll  # Guarda el array completo
                }
                track_id += 1

# Guardar el diccionario en un archivo pickle para cargarlo fácilmente luego
output_pickle = "dataset_dict.pkl"
with open(output_pickle, 'wb') as f:
    pickle.dump(dataset_dict, f)

print(f"Dataset guardado correctamente en {output_pickle}")


ModuleNotFoundError: No module named 'pretty_midi'

In [ ]:
import pickle

# Cargar el dataset desde el archivo pickle
with open("dataset_dict.pkl", "rb") as f:
    dataset_dict = pickle.load(f)

# Ejemplo: mostrar información del primer track
first_track = dataset_dict[0]
print("Artista:", first_track['artist'])
print("Track:", first_track['track'])
print("Compositor:", first_track['composer'])
print("Tonalidad:", first_track['key'])
print("Path MIDI:", first_track['midi_path'])
print("Forma del piano roll:", first_track['piano_roll'].shape)
